In [ ]:
# Cài đặt các thư viện cần thiết để chạy chương trình
%tensorflow_version 1.x
!pip install pyvi
!pip install ftfy
!pip install gensim

In [ ]:
# Đường dẫn được lưu trên google drive
from google.colab import drive
drive.mount('/content/gdrive')
path_train ='/content/gdrive/My Drive/ABSA/Dataset/Train.txt'
path_dev ='/content/gdrive/My Drive/ABSA/Dataset/Dev.txt'
path_test ='/content/gdrive/My Drive/ABSA/Dataset/Test.txt'


# Các hàm embedding khác nhau theo thử nghiệm của bài báo
embedding_path = '/content/gdrive/My Drive/ABSA/Embedding/W2V_ner.vec'
#embedding_path = '/content/gdrive/My Drive/ABSA/Embedding/Bert_Base_ner.vec'
#embedding_path = '/content/gdrive/My Drive/ABSA/Embedding/ELMO_ner.vec'
#embedding_path = '/content/gdrive/My Drive/ABSA/Embedding/MULTI_W_F_B_E.vec'
#embedding_path = '/content/gdrive/My Drive/ABSA/Embedding/W2V_C2V_ner.vec'

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import tensorflow as tf
import os, pickle, re, keras, sklearn, string
from keras.callbacks import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.models import load_model
from pyvi import ViTokenizer, ViPosTagger
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.utils.vis_utils import plot_model
import operator

In [ ]:
# Hàm đọc pretrained embedding 
import io
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    count_vocab = 0
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        coefs = np.asarray(tokens[1:], dtype='float64')
        data[tokens[0]] = coefs
        count_vocab += 1
    print("Tổng số các từ vựng là: " + str(count_vocab))
    return data

word_embedding = load_vectors(embedding_path)
EMBEDDING_DIM = word_embedding['yêu'].shape[0]
print("Chiều của embedding : ", EMBEDDING_DIM)

In [ ]:
class Review():
    def __init__(self,stt,text,label):
        self.stt = stt
        self.labels = label
        self.orginalText = text

In [ ]:
# Các hàm tiền xử lý dữ liệu
def normalText(sent):
    sent = str(sent).replace('_',' ').replace('/',' trên ')
    sent = re.sub('-{2,}','',sent)
    sent = re.sub('\\s+',' ', sent)
    patPrice = r'([0-9]+k?(\s?-\s?)[0-9]+\s?(k|K))|([0-9]+(.|,)?[0-9]+\s?(triệu|ngàn|trăm|k|K|))|([0-9]+(.[0-9]+)?Ä‘)|([0-9]+k)'
    patURL = r"(?:http://|www.)[^\"]+"
    sent = re.sub(patURL,'website',sent)
    sent = re.sub(patPrice, ' giá_tiền ', sent)
    sent = re.sub('\.+','.',sent)
    sent = re.sub('(hagtag\\s+)+',' hagtag ',sent)
    sent = re.sub('\\s+',' ',sent)
    return sent

def deleteIcon(text):
    text = text.lower()
    s = ''
    pattern = r"[a-zA-ZaăâbcdđeêghiklmnoôơpqrstuưvxyàằầbcdđèềghìklmnòồờpqrstùừvxỳáắấbcdđéếghíklmnóốớpqrstúứvxýảẳẩbcdđẻểghỉklmnỏổởpqrstủửvxỷạặậbcdđẹệghịklmnọộợpqrstụựvxỵãẵẫbcdđẽễghĩklmnõỗỡpqrstũữvxỹAĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXYÀẰẦBCDĐÈỀGHÌKLMNÒỒỜPQRSTÙỪVXỲÁẮẤBCDĐÉẾGHÍKLMNÓỐỚPQRSTÚỨVXÝẠẶẬBCDĐẸỆGHỊKLMNỌỘỢPQRSTỤỰVXỴẢẲẨBCDĐẺỂGHỈKLMNỎỔỞPQRSTỦỬVXỶÃẴẪBCDĐẼỄGHĨKLMNÕỖỠPQRSTŨỮVXỸ,._]"
    
    for char in text:
        if char !=' ':
            if len(re.findall(pattern, char)) != 0:
                s+=char
            elif char == '_':
                s+=char
        else:
            s+=char
    s = re.sub('\\s+',' ',s)
    return s.strip()


def clean_doc(doc):
    for punc in string.punctuation:
        doc = doc.replace(punc,' '+ punc + ' ')
    doc = normalText(doc)
    doc = deleteIcon(doc)
    doc = ViTokenizer.tokenize(doc)
    # Lowercase
    doc = doc.lower()
    # Removing multiple whitespaces
    doc = re.sub(r"\?", " \? ", doc)
    # Remove numbers
    doc = re.sub(r"[0-9]+", " num ", doc)
    # Split in tokens
    # Remove punctuation
    for punc in string.punctuation:
      if punc not in "_":
          doc = doc.replace(punc,' ')
    doc = re.sub('\\s+',' ',doc)
    return doc

In [ ]:
# hàm đọc dữ liệu lên từ file .txt theo định dạng của dữ liệu
from ftfy import fix_text
def read_files(filename):
    documents = list()
    count = 0
    with open(filename,'r',encoding='utf8') as file:
        text1 = file.read()
        for item in text1.split('\n'):
            if item != '':
                item = item.strip()
                if count == 0:
                    stt = fix_text(item)
                    count += 1
                elif count == 1:
                    text = fix_text(item)
                    count += 1
                elif count == 2:
                    labels = fix_text(item)
                    count = 0
                    reviewtemp = Review(stt, text, labels)
                    documents.append(reviewtemp)
    return documents

In [ ]:
# Hàm chuyển các nhãn thành các vector đưa vào mô hình máy học
def to_category_vector(label):
    vector = np.zeros(4).astype(np.float64)
    if label.strip() != '':
        if 'positive' in label:
            vector[1] = 1.0
        elif 'neutral' in label:
            vector[2] = 1.0
        elif 'negative' in label:
            vector[3] = 1.0
        else:
            vector[0] = 1.0
    else:
        vector[0] = 1.0
    return vector

In [ ]:
# Đoạn code này sẽ chuyển tất cả nhãn thành một danh sách các vector biểu diễn tương ứng
import re
def create_output(labels,categories,list_output):
    for i,category in enumerate(categories):
        if category in labels:
            try:
                output = re.findall('('+category+',\s(positive|negative|neutral))',labels)[0][0]
            except:
                output = ''
            list_output[i].append(to_category_vector(output))
        else:
            list_output[i].append(to_category_vector(''))
    return list_output

In [ ]:
# Đoạn code đọc dữ liệu từ các file .txt của tập dữ liệu train, dev, test. 
# Sau đó chuyển nhãn dưới dạng text thành dạng vector lưu vào biến.

listLabel = 'FOOD#STYLE&OPTIONS,FOOD#PRICES,FOOD#QUALITY,DRINKS#STYLE&OPTIONS,DRINKS#QUALITY,DRINKS#PRICES,RESTAURANT#GENERAL,RESTAURANT#MISCELLANEOUS,RESTAURANT#PRICES,LOCATION#GENERAL,SERVICE#GENERAL,AMBIENCE#GENERAL'
categories = listLabel.split(',')
print(len(categories))
document_X_train = []
document_Y_train = []
document_X_dev = []
document_Y_dev = []
document_X_test = []
document_Y_test = []

output_train1 = []
output_train1 = []
output_train2 = []
output_train3 = []
output_train4 = []
output_train5 = []
output_train6 = []
output_train7 = []
output_train8 = []
output_train9 = []
output_train10 = []
output_train11 = []
output_train12 = []

list_output_train = list()
list_output_train.append(output_train1)
list_output_train.append(output_train2)
list_output_train.append(output_train3)
list_output_train.append(output_train4)
list_output_train.append(output_train5)
list_output_train.append(output_train6)
list_output_train.append(output_train7)
list_output_train.append(output_train8)
list_output_train.append(output_train9)
list_output_train.append(output_train10)
list_output_train.append(output_train11)
list_output_train.append(output_train12)

for review in read_files(path_train):
    try:
        document_X_train.append(review.orginalText)
        list_output_train = create_output(review.labels, categories,list_output_train)
    except:
        print(review.labels)
        print(review.orginalText)

output_dev1 = []
output_dev2 = []
output_dev3 = []
output_dev4 = []
output_dev5 = []
output_dev6 = []
output_dev7 = []
output_dev8 = []
output_dev9 = []
output_dev10 = []
output_dev11 = []
output_dev12 = []
list_output_dev = list()
list_output_dev.append(output_dev1)
list_output_dev.append(output_dev2)
list_output_dev.append(output_dev3)
list_output_dev.append(output_dev4)
list_output_dev.append(output_dev5)
list_output_dev.append(output_dev6)
list_output_dev.append(output_dev7)
list_output_dev.append(output_dev8)
list_output_dev.append(output_dev9)
list_output_dev.append(output_dev10)
list_output_dev.append(output_dev11)
list_output_dev.append(output_dev12)
for review in read_files(path_dev):
    document_X_dev.append(review.orginalText)
    list_output_dev = create_output(review.labels, categories,list_output_dev)

for review in read_files(path_test):
    document_X_test.append(review.orginalText)    

In [ ]:
# Chạy hàm tiền xử lý dữ liệu (clean_doc) trên toàn bộ tập dữ liệu train và tập dữ liệu test.
totalX_train = []
totalY_train = np.array(document_Y_train)

totalX_test = []
totalY_test = np.array(document_Y_test)

totalX_dev = []
totalY_dev = np.array(document_Y_dev)

for i, doc in enumerate(document_X_train):
    totalX_train.append(clean_doc(doc))
for i, doc in enumerate(document_X_test):
    totalX_test.append(clean_doc(doc))

for i, doc in enumerate(document_X_dev):
    totalX_dev.append(clean_doc(doc))
print(totalX_train[0])
print(totalX_train[1])
print(totalX_train[2])

In [ ]:
# Đây là hàm chuyển các dữ liệu văn bản đã được tiền xử lý tạo thành vector số để đưa vào từ điển.
# Ý tưởng là : sẽ tạo thành 1 tập từ điển, mỗi từ sẽ có 1 chỉ số index trong từ điển, chúng ta sẽ thay từ đó bằng vị trí của nó trong từ điển
# Bởi vì giá trị input của mô hình là cố định nên chúng ta sẽ lấy câu dài nhất là đầu vào mô hình.
# Những câu nào ngắn hơn thì sẽ tự động thêm số 0 để cho đủ độ dài.
xLengths = [len(x.split(' ')) for x in totalX_train]
h = sorted(xLengths)  #sorted lengths
maxLength =h[len(h)-1]
print("max input length is: ",maxLength)
input_tokenizer = Tokenizer(filters="",oov_token="UNK")
input_tokenizer.fit_on_texts(totalX_train)
input_vocab_size = len(input_tokenizer.word_index) + 1
word_index = input_tokenizer.word_index
print("input_vocab_size:",input_vocab_size)
totalX = np.array(pad_sequences(input_tokenizer.texts_to_sequences(totalX_train), maxlen=maxLength,padding="post"))
textArray_dev = np.array(pad_sequences(input_tokenizer.texts_to_sequences(totalX_dev), maxlen=maxLength,padding="post"))
print(totalX_train[0])
print(totalX[0])


In [ ]:
# Hàm khởi tạo giá trị embedding được lấy từ pretrained word embedding.
# Nghĩa các tập từ vựng thì từ nào có trong bộ pre-trained embedding sẽ thay thế còn không có thì sẽ khởi tọa là vector 0.
def generate_embedding(word_index, model_embedding,EMBEDDING_DIM):
    count6 = 0
    countNot6 = 0
    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM)) 
    list_oov = []
    word_is_trained = []
    for word, i in word_index.items():
        try:
            embedding_vector = model_embedding[word]
            word_is_trained.append(word)
        except:
            list_oov.append(word)
            countNot6 +=1
            continue
        if embedding_vector is not None:
            count6 +=1
            embedding_matrix[i] = embedding_vector
    
    print('Number of words in pre-train embedding: ' + str(count6))
    print('Number of words not in pre-train embedding: ' + str(countNot6))
    print(list_oov)
    return embedding_matrix,word_is_trained

In [ ]:
# Chạy hàm khởi tạo embedding và hiện thị các từ có trong embedding.
embedding_matrix,word_is_trained = generate_embedding(word_index,word_embedding,EMBEDDING_DIM)
print(len(word_is_trained))


In [ ]:
# Khởi tạo mô hình CNN 
import pickle
he_normal_initializer = keras.initializers.he_normal(seed=1234)
glorot_normal_initializer = keras.initializers.glorot_uniform(seed=1995)
recurrent_units = 128 # best 64
filter_nums = 128 # best 128


def build_model3(sequence_max_length,EMBEDDING_DIM):
        inputs  = Input(shape=(sequence_max_length, ), dtype='float64', name='inputs')    
        embedding_layer = Embedding(input_vocab_size,EMBEDDING_DIM,input_length=sequence_max_length, trainable=True, weights=[embedding_matrix],name = 'word_emb_DOMAIN')(inputs)
        
        conv_1 = Conv1D(filter_nums, 3, kernel_initializer=he_normal_initializer, padding="same", activation="relu")(embedding_layer)
        conv_2 = Conv1D(filter_nums, 4, kernel_initializer=he_normal_initializer, padding="same", activation="relu")(embedding_layer)
        conv_3 = Conv1D(filter_nums, 5, kernel_initializer=he_normal_initializer, padding="same", activation="relu")(embedding_layer)
        
        maxpool_1 = GlobalMaxPooling1D()(conv_1)
        maxpool_2 = GlobalMaxPooling1D()(conv_2)
        maxpool_3 = GlobalMaxPooling1D()(conv_3)
        v0_col = Concatenate(axis=1)([maxpool_1, maxpool_2, maxpool_3])     
         
        fc2 = Dropout(0.2)(Dense(300, name = 'dense_2',trainable=True)(v0_col))
         
        output1 = Dense(4,name="output1", activation='softmax')(fc2)
        output2 = Dense(4,name="output2", activation='softmax')(fc2)
        output3 = Dense(4,name="output3", activation='softmax')(fc2)
        output4 = Dense(4,name="output4", activation='softmax')(fc2)
        output5 = Dense(4,name="output5", activation='softmax')(fc2)
        output6 = Dense(4,name="output6", activation='softmax')(fc2)
        output7 = Dense(4,name="output7", activation='softmax')(fc2)
        output8 = Dense(4,name="output8", activation='softmax')(fc2)
        output9 = Dense(4,name="output9", activation='softmax')(fc2)
        output10 = Dense(4,name="output10", activation='softmax')(fc2)
        output11 = Dense(4,name="output11", activation='softmax')(fc2)
        output12 = Dense(4,name="output12", activation='softmax')(fc2)
        

        model = Model(inputs=inputs, outputs=[output1,output2,output3,output4,output5,output6,output7,output8,output9,output10,output11,output12])
        
        tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
        
        list_loss = ['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy']
        model.compile(loss = list_loss, optimizer="adam", metrics=['accuracy'])

        list_total_Y_train  = [np.array(list_output_train[0]),np.array(list_output_train[1]),np.array(list_output_train[2]),np.array(list_output_train[3]),np.array(list_output_train[4]),np.array(list_output_train[5]),np.array(list_output_train[6]),np.array(list_output_train[7]),np.array(list_output_train[8]),np.array(list_output_train[9]),np.array(list_output_train[10]),np.array(list_output_train[11])]
        list_total_Y_dev = [np.array(list_output_dev[0]),np.array(list_output_dev[1]),np.array(list_output_dev[2]),np.array(list_output_dev[3]),np.array(list_output_dev[4]),np.array(list_output_dev[5]),np.array(list_output_dev[6]),np.array(list_output_dev[7]),np.array(list_output_dev[8]),np.array(list_output_dev[9]),np.array(list_output_dev[10]),np.array(list_output_dev[11])]       
        history = model.fit(totalX, list_total_Y_train, validation_data = [textArray_dev,list_total_Y_dev],batch_size=50, epochs=100,callbacks=[tensorBoardCallback])
        plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
        return model

model = build_model3(maxLength,EMBEDDING_DIM)

model.save("ResCNN_model.h5")

In [ ]:
# Hàm dự đoán trên dữ liệu tập test.
textArray_test = np.array(pad_sequences(input_tokenizer.texts_to_sequences(totalX_test), maxlen=maxLength,padding="post"))
test_length = len(totalX_test)

print(totalX_test[0])
print(textArray_test[0])
count = 0
textPrint = ''
textREsult = []
for iz,item in enumerate(textArray_test):
    predicted = model.predict([np.expand_dims(item, axis=0)])
    s = ''
    for i, predict in enumerate(predicted):
        index2, value = max(enumerate(predict[0]), key=operator.itemgetter(1))
        if index2 == 1:
            s+= '{' + str(categories[i]) + ', positive}, '
        elif index2 == 2:
            s+= '{' + str(categories[i]) + ', neutral}, '
        elif index2 == 3:
            s+= '{' + str(categories[i]) + ', negative}, '
    textPrint += '#' + str(count +1)+'\n'
    textPrint += document_X_test[iz] + '\n'
    textPrint += s[:len(s)-2] +'\n\n'
    count +=1
with open('CNN.txt','w',encoding = 'utf8') as file:
    file.write(textPrint)

In [ ]:
# Hàm dự đoán trên 1 dữ liệu đầu vào
input_text = "- Mùa đông rồi còn gì tuyệt vời hơn là ăn ốc với ăn ngao 😍 - Nguyễn Khiết là ngõ cắt ngang Phúc Tân ấy các bạn ạ. Chỗ mà nhiều nhiều hàng karaoke. - Ăn ở đây thì hơi xa một chút nhưng ôii thôi ngon thôi rồi đảm bảo không mất công lên đây ăn đâu. - Giá chỉ 30k-40k cho 1 bát ngao hấp đầy ú ụ luôn 💓- Chỗ như ảnh mình ăn hết có 100k huhu mà 2 ng ăn no chết đi đc. Coca có 5k 1 chai trà đá thì free luôn nhé các bạn 😍 - Bác chủ quán hiền lắm luôn ý. Nhiều lúc mình gọi nhiều ăn chả hết xong bác ý còn chả lấy tiền cơ 😭😩"
input_clean = clean_doc(input_text)
input_index = np.array(pad_sequences(input_tokenizer.texts_to_sequences([input_clean]), maxlen=maxLength,padding="post"))

predicted = model.predict([np.expand_dims(input_index[0], axis=0)])
output = ''
for i, predict in enumerate(predicted):
    index2, value = max(enumerate(predict[0]), key=operator.itemgetter(1))
    if index2 == 1:
        output+= '{' + str(categories[i]) + ', positive}, '
    elif index2 == 2:
        output+= '{' + str(categories[i]) + ', neutral}, '
    elif index2 == 3:
        output+= '{' + str(categories[i]) + ', negative}, '
output = output[:-2]
print("Đầu vào: ", input_text)
print("Dự đoán: ", output)